In [2]:
import pandas as pd
import glob
import re


file_paths = glob.glob("metro/data/*.csv")
# 노선명 변환 사전
line_name_mapping = {
    '경인선': '1호선',
    '과천선': '4호선',
    '안산선': '4호선',
    '일산선': '3호선',
    '장항선': '1호선',
    '공항철도 1호선': '공항철도',
    '9호선2~3단계': '9호선',
    '수인선': '수인분당선',
    '분당선': '수인분당선',
    '경의선': '경의중앙선',
    '중앙선': '경의중앙선',
    '경부선': '1호선'
}

# 경원선 노선명 재할당 함수
def reassign_kyungwon_line(row):
    if row['노선명'] == '경원선':
        station = row['역명']
        if station in ["연천", "전곡", "청산", "소요산", "동두천", "보산", "동두천중앙", "지행", "덕정", "덕계", "양주", "녹양", "가능", "의정부", "회룡", "망월사", "도봉산", "도봉", "방학", "창동", "녹천", "월계", "광운대", "석계", "신이문", "외대앞", "회기"]:
            return '1호선'
        elif station in ["용산", "이촌", "서빙고", "한남", "옥수", "응봉", "왕십리", "청량리"]:
            return '경의중앙선'
        elif station == '왕십리':
            return '수인분당선'
        else:
            return '알 수 없는 노선'
    else:
        return row['노선명']


def clean_station_name(station_name):
    # 소괄호로 둘러싸인 텍스트 제거
    cleaned_name = re.sub(r'\([^)]*\)', '', station_name)
    # 띄어쓰기 제거
    cleaned_name = cleaned_name.replace(' ', '')
    
    return cleaned_name

# CSV 파일 읽어서 변환 및 머징
merged_df = pd.DataFrame()

for file_path in file_paths:
    df = pd.read_csv(file_path)
    df['역명'] = df['역명'].apply(clean_station_name)
    df['노선명'] = df['노선명'].replace(line_name_mapping)
    df['노선명'] = df.apply(reassign_kyungwon_line, axis=1)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

merged_df = merged_df.rename(columns={"사용일자": "일자"})

# 기후동행카드 기준 일자 설정
cutoff_date = 20240127

# 'after' 컬럼 추가
merged_df['after'] = merged_df['일자'].apply(lambda x: 1 if int(x) >= cutoff_date else 0)

print(merged_df)


             일자    노선명        역명  승차총승객수  하차총승객수  after
0      20231101    8호선     단대오거리   13482   13072      0
1      20231101  경의중앙선        원덕     372     362      0
2      20231101  경의중앙선        용문    2194    2097      0
3      20231101  경의중앙선        지평      72      71      0
4      20231101  우이신설선  4.19민주묘지    4515    4447      0
...         ...    ...       ...     ...     ...    ...
93151  20240331    4호선        초지    3256    3187      1
93152  20240331    4호선        안산   12223   10969      1
93153  20240331    4호선      신길온천    1692    1497      1
93154  20240331    4호선        정왕    7212    7638      1
93155  20240331    4호선       오이도    6545    6253      1

[93156 rows x 6 columns]


In [4]:
# 기후동행카드 컬럼 할당 함수
def assign_category(row):
    line = row['노선명']
    station = row['역명']
    
    if line in ["2호선", "6호선", "7호선", "8호선", "9호선"]:
        return '승하차 가능'
    elif line in ["신림선", "우이신설선", "김포골드라인", "경강선"]:
        return '승하차 불가능'
    elif line == '서해선':
        return '승차 불가능 / 하차 가능' if station == '부천종합운동장' else '승하차 불가능'
    elif line == '1호선':
        restricted_stations = ["연천", "전곡", "청산", "소요산", "동두천", "보산", "동두천중앙", "지행", "덕정", "덕계", "양주", "녹양", "가능", "의정부", "회룡", "망월사", "인천", "동인천", "도원", "제물포", "도화", "주안", "간석", "동암", "백운", "부평", "부개", "송내", "중동", "부천", "소사", "역곡", "신창", "온양온천", "배방", "탕정", "아산", "쌍용", "봉명", "천안", "두정", "직산", "성환", "평택", "평택지제", "서정리", "송탄", "진위", "오산", "오산대", "세마", "병점", "서동탄", "세류", "수원", "화서", "성균관대", "의왕", "당정", "군포", "금정", "명학", "안양", "관악", "석수", "광명"]
        return '승하차 불가능' if station in restricted_stations else '승하차 가능'
    elif line == '3호선':
        restricted_stations = ["대화", "주엽", "정발산", "마두", "백석", "대곡", "화정", "원당", "원흥", "삼송"]
        return '승하차 불가능' if station in restricted_stations else '승하차 가능'
    elif line == '4호선':
        restricted_stations = ["오이도", "정왕", "신길온천", "안산", "초지", "고잔", "중앙", "한대앞", "상록수", "반월", "대야미", "수리산", "산본", "금정", "범계", "평촌", "인덕원", "정부과천청사", "과천", "대공원", "경마공원", "선바위"]
        special_stations = ["진접", "오남", "별내별가람"]
        if station in restricted_stations:
            return '승하차 불가능'
        elif station in special_stations:
            return '승차 불가능 / 하차 가능'
        else:
            return '승하차 가능'
    elif line == '5호선':
        restricted_stations = ["하남검단산", "하남시청", "하남풍산", "미사"]
        return '승차 불가능 / 하차 가능' if station in restricted_stations else '승하차 가능'
    elif line == '경의중앙선':
        restricted_stations = ["임진강", "운천", "문산", "파주", "월롱", "금촌", "금릉", "운정", "야당", "탄현", "일산", "풍산", "백마", "곡산", "대곡", "능곡", "행신", "강매", "한국항공대", "지평", "용문", "원덕", "양평", "오빈", "아신", "국수", "신원", "양수", "운길산", "팔당", "도심", "덕소", "양정", "도농", "구리"]
        return '승하차 불가능' if station in restricted_stations else '승하차 가능'
    elif line == '수인분당선':
        restricted_stations = ["인천", "신포", "숭의", "인하대", "송도", "연수", "원인재", "남동인더스파크", "호구포", "인천논현", "소래포구", "월곶", "달월", "오이도", "정왕", "신길온천", "안산", "초지", "고잔", "중앙", "한대앞", "사리", "야목", "어천", "오목천", "고색", "수원", "매교", "수원시청", "매탄권선", "망포", "영통", "청명", "상갈", "기흥", "신갈", "구성", "보정", "죽전", "오리", "미금", "정자", "수내", "서현", "이매", "야탑", "태평", "가천대"]
        return '승하차 불가능' if station in restricted_stations else '승하차 가능'
    elif line == '경춘선':
        restricted_stations = ["춘천", "남춘천", "김유정", "강촌", "백양리", "굴봉산", "가평", "상천", "청평", "대성리", "마석", "천마산", "평내호평", "금곡", "사릉", "퇴계원", "별내", "갈매"]
        return '승하차 불가능' if station in restricted_stations else '승하차 가능'
    elif line == '공항철도':
        restricted_stations = ["인천공항2터미널", "인천공항1터미널", "공항화물청사", "운서", "영종", "청라국제도시", "검암", "계양"]
        return '승하차 불가능' if station in restricted_stations else '승하차 가능'
    else:
        return '알 수 없음'

# 새로운 카테고리컬 컬럼 할당
merged_df['기후동행카드'] = merged_df.apply(assign_category, axis=1)

merged_df.to_csv('metro/preprocessed_data/metro_기후동행카드(노선+역).csv', encoding='utf-8-sig', index=False)

In [6]:
def aggregate_with_consistent_category(df):
    grouped = df.groupby(['일자', 'after', '역명'])
    aggregated = grouped.agg({
        '승차총승객수': 'sum',
        '하차총승객수': 'sum',
        '기후동행카드': lambda x: x.iloc[0] if (x == x.iloc[0]).all() else '불일치'
    }).reset_index()
    return aggregated[aggregated['기후동행카드'] != '불일치']

aggregated_df = aggregate_with_consistent_category(merged_df.drop(columns=['노선명']))
aggregated_df.to_csv('metro/preprocessed_data/metro_기후동행카드(역).csv', encoding='utf-8-sig', index=False)

In [9]:

import pandas as pd

df = pd.read_csv('metro/preprocessed_data/metro_기후동행카드(노선+역).csv')

df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')

# 20231104 이후의 데이터만 사용
df = df[df['일자'] >= '2023-11-04']

# 20240329 이전의 데이터만 사용
df = df[df['일자'] <= '2024-03-29']

# 일주일 단위 인덱스 생성 (2023-11-04 기준)
start_date = pd.to_datetime('2023-11-04')
df['week'] = ((df['일자'] - start_date).dt.days // 7) + 1

# 그룹화 및 평균 계산
preprocessed_data = df.groupby(['week', 'after', '노선명', '역명', '기후동행카드']).agg({
    '승차총승객수': 'mean',
    '하차총승객수': 'mean'
}).reset_index()

preprocessed_data.to_csv('metro/preprocessed_data/metro_기후동행카드(노선+역_week).csv', encoding='utf-8-sig', index=False)

In [4]:

import pandas as pd

df = pd.read_csv('metro/preprocessed_data/metro_기후동행카드(역).csv')

df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')

# 20231104 이후의 데이터만 사용
df = df[df['일자'] >= '2023-11-04']

# 20240329 이전의 데이터만 사용
df = df[df['일자'] <= '2024-03-29']

# 일주일 단위 인덱스 생성 (2023-11-04 기준)
start_date = pd.to_datetime('2023-11-04')
df['week'] = ((df['일자'] - start_date).dt.days // 7) + 1

# 그룹화 및 평균 계산
preprocessed_data = df.groupby(['week', 'after', '역명', '기후동행카드']).agg({
    '승차총승객수': 'mean',
    '하차총승객수': 'mean'
}).reset_index()

preprocessed_data

,week,after,역명,기후동행카드,승차총승객수,하차총승객수
0,1,0,4.19민주묘지,승하차 불가능,3790.000000,3568.428571
1,1,0,가능,승하차 불가능,6427.000000,6163.428571
2,1,0,가락시장,승하차 가능,16653.428571,17536.571429
3,1,0,가산디지털단지,승하차 가능,55514.000000,58005.285714
4,1,0,가양,승하차 가능,22097.714286,21480.857143
...,...,...,...,...,...,...
10845,21,1,회기,승하차 가능,30311.571429,29205.142857
10846,21,1,회룡,승하차 불가능,13369.428571,13027.142857
10847,21,1,회현,승하차 가능,27917.000000,29570.000000
10848,21,1,효창공원앞,승하차 가능,11369.857143,11056.571429


In [6]:
preprocessed_data.to_csv('metro/preprocessed_data/metro_기후동행카드(역_week).csv', encoding='utf-8-sig', index=False)

In [4]:
import pandas as pd

weekly_users_df = pd.read_csv('weekly_users(interpolated).csv')
metro_df = pd.read_csv('metro/preprocessed_data/metro_기후동행카드(역_week).csv')

In [5]:
# 필요한 컬럼과 키 컬럼만 포함한 데이터프레임 생성
weekly_users_df = weekly_users_df[['average_users', 'week']]

# 두 데이터프레임을 키 컬럼을 기준으로 병합
merged_df = pd.merge(metro_df, weekly_users_df, on='week', how='left')

# 병합된 데이터프레임 저장
merged_df.to_csv('metro/preprocessed_data/metro_기후동행카드(역_week_number_of_users).csv', index=False)